### Bronze Layer: Ingest results.json

Load raw JSON from landing zone and save as parquet in bronze/raw layer.

In [1]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

/home/ricardo/Documentos/Projects/dab-ci-cd/dab_project/.venv/lib/python3.12/site-packages/databricks/sdk/_widgets/__init__.py:71: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import current_timestamp, lit

In [3]:
from formula1.formula1_constants import landing_folder_path, raw_folder_path

##### Step 1 - Define schema and read JSON from landing zone

In [4]:
results_schema = StructType(fields=[
    StructField("resultId", IntegerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("grid", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("positionText", StringType(), True),
    StructField("positionOrder", IntegerType(), True),
    StructField("points", FloatType(), True),
    StructField("laps", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("fastestLap", IntegerType(), True),
    StructField("rank", IntegerType(), True),
    StructField("fastestLapTime", StringType(), True),
    StructField("fastestLapSpeed", FloatType(), True),
    StructField("statusId", StringType(), True)
])

In [5]:
results_df = spark.read \
    .schema(results_schema) \
    .json(f"{landing_folder_path}/results.json")

##### Step 2 - Add ingestion metadata

In [6]:
results_df = results_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to bronze/raw layer as parquet

In [7]:
results_df.write.mode("overwrite").parquet(f"{raw_folder_path}/results")